# 1. 카카오톡 대화 데이터 전처리

In [12]:
import openai
from openai import OpenAI
import json
from dotenv import load_dotenv
import os

# 🔑 OpenAI API 키 설정
load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

client = OpenAI()

# 📂 고객 대화 데이터 로드 (merged_by_client.json)
with open("lieu_consulting_merged.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [35]:
def extract_problem_from_customer_dialogue(client_data):
    dialogue = ""
    for pair in client_data["messages"]:
        dialogue += f"Q: {pair['question']}\nA: {pair['answer']}\n"

    prompt = f"""다음은 한 고객과 전문가 Lieu 사이의 상담 대화입니다:

{dialogue}

위 대화를 참고하여 이 고객의 '문제점'을 요약해주세요.
치료, 통증 등 의료에 관한 문제가 아니라면 "문제없음" 이라고 말해주세요.
의료에 관한 문제라면 아래 예시처럼 JSON 형태로 데이터를 반환해 주세요.

<예시데이터>
{{
    "문제 종류" : "내전근 통증", 
    "문제 내용" : "고객은 2주 동안 축구를 하다 스프린트를 시작할 때 왼쪽 내전근에 심한 통증을 겪었습니다. 아이싱과 마사지를 했으나, 통증은 계속되고 있습니다."
}}
"""

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content.strip()

In [37]:
# ✅ 실행 예시 (첫 번째 고객만)
result = extract_problem_from_customer_dialogue(data[1])
print(f"Client: {data[1]['client']}\nProblem: {result}")

Client: 아킬레스
Problem: {
    "문제 종류" : "아킬레스 통증", 
    "문제 내용" : "고객은 아킬레스 통증 관리를 위해 가장 빠른 회복 방법과 세미나에 관한 정보를 찾고 있습니다."
}


In [38]:
from tqdm import tqdm

kakao_data = []

for item in tqdm(data):
    result = extract_problem_from_customer_dialogue(item)

    if result == "문제없음":
        continue
    
    clean_data = {"problem" : result, "solution": ""}
    kakao_data.append(clean_data)

    print(f"✅ Client: {item['client']}\nProblem: {result}")


  2%|▏         | 1/55 [00:05<04:35,  5.11s/it]

✅ Client: .
Problem: {
    "문제 종류" : "내전근 통증", 
    "문제 내용" : "고객은 2주 동안 축구를 하다 스프린트를 시작할 때 왼쪽 내전근에 심한 통증을 겪었습니다. 아이싱과 마사지를 했으나, 통증은 계속되고 있습니다."
}


  4%|▎         | 2/55 [00:10<04:37,  5.23s/it]

✅ Client: 아킬레스
Problem: {
    "문제 종류" : "아킬레스 통증", 
    "문제 내용" : "고객은 아킬레스 통증 관리에 대한 방법을 알고 싶어하고, 회복을 가속화 할 수 있는 다양한 방법을 알아보고자 합니다."
}


  7%|▋         | 4/55 [00:18<04:03,  4.78s/it]

...

✅ Client: 베개를 부비적대는 라이언
Problem: "문제없음"


 98%|█████████▊| 54/55 [03:19<00:04,  4.28s/it]

✅ Client: 박승민
Problem: {
    "문제 종류" : "운동 제한", 
    "문제 내용" : "고객은 일정 범위 이상으로 운동할 수 없는 제한이 있습니다. 보통 0도에서 -5도 정도가 정상 범위이지만 고객은 3~5도까지만 가능하며, 이로 인해 나중에 운동하거나 할 때 지장을 겪을까 염려하고 있습니다."
}


100%|██████████| 55/55 [03:27<00:00,  3.78s/it]

✅ Client: 베개를 부비적대는 라이언
Problem: {
    "문제 종류" : "몸의 초기화 및 통증 관리", 
    "문제 내용" : "현재 고객은 몸이 초기화되면 가진 통증들이 사라질지, 초기화 후에 운동이 가능한지, 그리고 치료 이전에 했던 축구와 런닝 등의 활동을 할 수 있는지 궁금해하고 있습니다. 또한 고객은 몸의 초기화를 위해 얼마나 많은 시간이 소요되는지, 예약 방법 및 치료 시간에 대한 정보를 원합니다. 구조적 문제에 대한 치료 가능성 또한 질문하였습니다."
}


In [41]:
import json
import pandas as pd

# 파싱된 결과 저장
parsed = []

for item in kakao_data:
    problem_raw = item["problem"]
    try:
        parsed_json = json.loads(problem_raw)
        if isinstance(parsed_json, dict):
            parsed.append({
                "문제 종류": parsed_json.get("문제 종류", ""),
                "문제 내용": parsed_json.get("문제 내용", ""),
                "solution": item["solution"]
            })
        else:
            parsed.append({
                "문제 종류": "없음",
                "문제 내용": str(parsed_json),
                "solution": item["solution"]
            })
    except json.JSONDecodeError:
        parsed.append({
            "문제 종류": "없음",
            "문제 내용": problem_raw.strip('"'),
            "solution": item["solution"]
        })

# DataFrame으로 변환 및 저장
df = pd.DataFrame(parsed)
df.head()


,문제 종류,문제 내용,solution
0,내전근 통증,고객은 2주 동안 축구를 하다 스프린트를 시작할 때 왼쪽 내전근에 심한 통증을 겪었...,
1,아킬레스 통증,"고객은 아킬레스 통증 관리에 대한 방법을 알고 싶어하고, 회복을 가속화 할 수 있는...",
2,왼쪽 고관절의 염증 및 힘줄 짧음,"고객은 왼쪽 고관절에 염증이 발생했으며, 엑스레이 결과 힘줄이 짧아진 상태로 나타났...",
3,무릎 통증과 반월판 부분파열,"고객은 무릎의 전체적인 통증이 심하며, 반월판에 부분파열이 일어났다고 합니다. 이에...",
4,재골절 및 발목 관련 통증,"고객은 작년 3월에 비골 골절로 수술을 하고, 11월에 철심 제거를 한 이후에 이번...",


In [44]:
clean_df = df[df["문제 종류"] != "없음"]
clean_df

,문제 종류,문제 내용,solution
0,내전근 통증,고객은 2주 동안 축구를 하다 스프린트를 시작할 때 왼쪽 내전근에 심한 통증을 겪었...,
1,아킬레스 통증,"고객은 아킬레스 통증 관리에 대한 방법을 알고 싶어하고, 회복을 가속화 할 수 있는...",
2,왼쪽 고관절의 염증 및 힘줄 짧음,"고객은 왼쪽 고관절에 염증이 발생했으며, 엑스레이 결과 힘줄이 짧아진 상태로 나타났...",
3,무릎 통증과 반월판 부분파열,"고객은 무릎의 전체적인 통증이 심하며, 반월판에 부분파열이 일어났다고 합니다. 이에...",
4,재골절 및 발목 관련 통증,"고객은 작년 3월에 비골 골절로 수술을 하고, 11월에 철심 제거를 한 이후에 이번...",
5,발목 상비골지대 봉합수술 후 회복,고객은 최근에 오른쪽 발목 상비골지대 봉합수술을 받았습니다. 그리고 현재는 재활이나...,
6,삼각인대 및 근육 통증,고객은 왼발 삼각인대와 오른쪽 사타구니 직근을 다쳐 킥이나 슈팅을 제대로 수행하지 ...,
8,"[아킬레스건염, 손목힘줄염]",고객은 아킬레스건염과 손목힘줄염 진단을 받았습니다. 손목 문제는 몇 개월 전부터 시...,
10,하체 통증,"고객은 2년 동안 하체 통증으로 고생하였으며, 매번 새로운 부위에서 통증이 발생했습...",
11,만성 통증,"고객의 남편이 운동을 하지는 않지만 종아리, 어깨, 목 부분에 매일 통증이 있다. ...",


In [46]:
clean_df.to_csv("kakao_clean_data.csv")

# 